In [0]:
%run "./Mini-Kurs-ETL-Spark/4. Scala/3. Pobierz Dane"

import java.nio.file.{Paths, Files}
file_exists: (path: String)Boolean

import sys.process._
import java.net.URL
import java.nio.file.{Files, Paths, StandardCopyOption}
getfile: (url: String, outputFile: String)Unit

actorsUrl: String = https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/actors.csv
moviesUrl: String = https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/movies.csv
namesUrl: String = https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/names.csv
ratingsUrl: String = https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/ratings.csv
filePath: String = /FileStore/tables/Files/
actorsFile: String = actors.csv
moviesFile: String = movies.csv
namesFile: String = names.csv
ratingsFile: String = ratings.csv
tmp: String = file:/tmp/
dbfsdestination: String = dbfs:/FileStore/tables/Files/

res0: AnyVal = ()

Zadanie 2

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("imdb_title_id", StringType(), False),
    StructField("ordering", IntegerType(), False),
    StructField("imdb_name_id", StringType(), False),
    StructField("category", StringType(), False),
    StructField("job", StringType(), False),
    StructField("characters", StringType(), False)
])

filePath = "dbfs:/FileStore/tables/Files/actors.csv"
df = spark.read.format("csv")\
.option("header", "true")\
.schema(schema)\
.load(filePath)

display(df.limit(10))
df.printSchema()

imdb_title_id,ordering,imdb_name_id,category,job,characters
tt0000009,1,nm0063086,actress,null,[Miss Geraldine Holbrook (Miss Jerry)]
tt0000009,2,nm0183823,actor,null,[Mr. Hamilton]
tt0000009,3,nm1309758,actor,null,[Chauncey Depew - the Director of the New York Central Railroad]
tt0000009,4,nm0085156,director,null,null
tt0000574,1,nm0846887,actress,null,[Kate Kelly]
tt0000574,2,nm0846894,actor,null,[School Master]
tt0000574,3,nm3002376,actor,null,[Steve Hart]
tt0000574,4,nm0170118,actress,null,null
tt0000574,5,nm0846879,director,null,null
tt0000574,6,nm0317210,producer,producer,null


root
 |-- imdb_title_id: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- imdb_name_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



Zadanie 3

In [0]:
# wczytanie surowego pliku 
file_path = "dbfs:/FileStore/tables/test-1.json"
df_raw = spark.read.json(file_path)
df_raw.show(truncate=False)
df_raw.printSchema()

+---------------+-------------+----------+--------+-----------+
|_corrupt_record|city         |date      |humidity|temperature|
+---------------+-------------+----------+--------+-----------+
|[              |null         |null      |null    |null       |
|null           |New York     |2024-03-18|60      |22         |
|null           |Los Angeles  |2024-03-18|55      |25         |
|null           |Chicago      |2024-03-18|70      |HOT        |
|null           |Miami        |2024-03-18|80      |null       |
|null           |San Francisco|2024-03-18|65      |18         |
|]              |null         |null      |null    |null       |
+---------------+-------------+----------+--------+-----------+

root
 |-- _corrupt_record: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- temperature: string (nullable = true)



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

file_path = "dbfs:/FileStore/tables/test-1.json"

schema = StructType([
    StructField("city", StringType(), True),
    StructField("temperature", IntegerType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("date", StringType(), True)
])

# read mode 1
df_permissive = spark.read.format("json").schema(schema).option("mode", "PERMISSIVE").load(file_path)
df_permissive.show()

+-------------+-----------+--------+----------+
|         city|temperature|humidity|      date|
+-------------+-----------+--------+----------+
|         null|       null|    null|      null|
|     New York|         22|      60|2024-03-18|
|  Los Angeles|         25|      55|2024-03-18|
|      Chicago|       null|      70|2024-03-18|
|        Miami|       null|      80|2024-03-18|
|San Francisco|         18|      65|2024-03-18|
|         null|       null|    null|      null|
+-------------+-----------+--------+----------+



In [0]:
# read mode 2
df_dropmalformed = spark.read.format("json").schema(schema).option("mode", "DROPMALFORMED").load(file_path)
df_dropmalformed.show()

+-----------+-----------+--------+----------+
|       city|temperature|humidity|      date|
+-----------+-----------+--------+----------+
|   New York|         22|      60|2024-03-18|
|Los Angeles|         25|      55|2024-03-18|
|      Miami|       null|      80|2024-03-18|
+-----------+-----------+--------+----------+



In [0]:
# read mode 3
try:
    df_failfast = spark.read.format("json").schema(schema).option("mode", "FAILFAST").load(file_path)
    df_failfast.show()
except Exception as e:
    print(f"FAILFAST ERROR: {e}")

FAILFAST ERROR: An error occurred while calling o871.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (ip-10-172-179-177.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/FileStore/tables/test.json.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:704)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:673)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:796)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:496)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.u

In [0]:
# read mode 4
bad_records_path = "dbfs:/tmp/badrecords"
df_bad_records = spark.read.format("json").schema(schema).option("badRecordsPath", bad_records_path).load(file_path)
df_bad_records.show()

+-----------+-----------+--------+----------+
|       city|temperature|humidity|      date|
+-----------+-----------+--------+----------+
|   New York|         22|      60|2024-03-18|
|Los Angeles|         25|      55|2024-03-18|
|      Miami|       null|      80|2024-03-18|
+-----------+-----------+--------+----------+



Zadanie 4

In [0]:
df_permissive.show()
df_permissive.printSchema()

+-------------+-----------+--------+----------+
|         city|temperature|humidity|      date|
+-------------+-----------+--------+----------+
|         null|       null|    null|      null|
|     New York|         22|      60|2024-03-18|
|  Los Angeles|         25|      55|2024-03-18|
|      Chicago|       null|      70|2024-03-18|
|        Miami|       null|      80|2024-03-18|
|San Francisco|         18|      65|2024-03-18|
|         null|       null|    null|      null|
+-------------+-----------+--------+----------+

root
 |-- city: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- date: string (nullable = true)



In [0]:
# zapis danych
df_permissive.write.mode("overwrite").json("dbfs:/FileStore/tables/output_json")
df_permissive.write.mode("overwrite").parquet("dbfs:/FileStore/tables/output_parquet")

In [0]:
# sprawdzenie zapisanych danych - json
df_json = spark.read.json("dbfs:/FileStore/tables/output_json")
df_json.show(truncate=False)
df_json.printSchema()
# dane są te same, kolumny zostały wczytane w innej kolejności

+-------------+----------+--------+-----------+
|city         |date      |humidity|temperature|
+-------------+----------+--------+-----------+
|null         |null      |null    |null       |
|New York     |2024-03-18|60      |22         |
|Los Angeles  |2024-03-18|55      |25         |
|Chicago      |2024-03-18|70      |null       |
|Miami        |2024-03-18|80      |null       |
|San Francisco|2024-03-18|65      |18         |
|null         |null      |null    |null       |
+-------------+----------+--------+-----------+

root
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- temperature: long (nullable = true)



In [0]:
# sprawdzenie zapisanych danych - parquet
df_parquet = spark.read.parquet("dbfs:/FileStore/tables/output_parquet")
df_parquet.show(truncate=False)
df_parquet.printSchema()

+-------------+-----------+--------+----------+
|city         |temperature|humidity|date      |
+-------------+-----------+--------+----------+
|null         |null       |null    |null      |
|New York     |22         |60      |2024-03-18|
|Los Angeles  |25         |55      |2024-03-18|
|Chicago      |null       |70      |2024-03-18|
|Miami        |null       |80      |2024-03-18|
|San Francisco|18         |65      |2024-03-18|
|null         |null       |null    |null      |
+-------------+-----------+--------+----------+

root
 |-- city: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- date: string (nullable = true)

